In [2]:

## Laden von der verschieden Bibliothek zur Daten Visualizierung und Vorhersagen
import os
import numpy as np
from matplotlib import pyplot
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression ,BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.optimizers import rmsprop
from keras import backend as ker
from math import sqrt
import tensorflow as tf
os.chdir('/home/jupyter/jupyterNotebooks/m4competition18/data')
os.getcwd()


/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'/home/jupyter/jupyterNotebooks/m4competition18/data'

In [3]:
df_hourly = pd.read_csv("Hourly-train.csv", skiprows=0, index_col =0)



Dataset_hourly = df_hourly.T.iloc[:,:]
print ("Dimension:  ",Dataset_hourly.shape,"\n")

Dimension:   (960, 414) 



In [10]:
## Laden von Dataset Weekly-train
df_weekly = pd.read_csv("Weekly-train.csv", skiprows=0, index_col =0)


Dataset_weekly = df_weekly.T
Weekly_train_Finance = df_weekly[59:223].T.iloc[:,:]
Weekly_train_Other = df_weekly[:12].T.iloc[:,:]
Weekly_train_Industry = df_weekly[53:59].T.iloc[:,:]
Weekly_train_Demographic = df_weekly[223:247].T.iloc[:,:]
Weekly_train_Micro = df_weekly[247:359].T.iloc[:,:]
Weekly_train_Macro = df_weekly[12:53].T.iloc[:,:]

print ("Dimension:  ",Dataset_weekly.shape,"\n")

Dimension:   (2597, 359) 



In [ ]:
## Laden von  Dataset Yearly-train
df_yearly = pd.read_csv("Yearly-train.csv", skiprows=0, index_col =0)


Dataset_yearly = df_yearly.T.iloc[:,:]
print ("Dimension:  ",Dataset_yearly.shape,"\n")


In [19]:
## Laden von Dataset Monthly-train
df_monthly = pd.read_csv("Monthly-train.csv", skiprows=0, index_col =0)
Dataset_monthly = df_monthly.T.iloc[:,:]
print ("Dimension:  ",Dataset_monthly.shape,"\n")


Dimension:   (2794, 48000) 



In [ ]:
## Laden von Dataset Quarterly-train
df_quaterly = pd.read_csv("Quarterly-train.csv", skiprows=0, index_col =0)
Dataset_quaterly = df_quaterly.T.iloc[:,:]
print ("Dimension:  ",Dataset_quaterly.shape,"\n")

In [ ]:
df_daily = pd.read_csv("Daily-train.csv", skiprows=0, index_col =0)
Dataset_daily = df_daily.T.iloc[:,:]
print ("Dimension:  ",Dataset_daily.shape,"\n")

In [6]:
def remov_nan (dataset):
    '''
    to remove all NaN Values in a 
    Time Serie Dataframe
    '''
    n = dataset.isnull().sum() 
    data = dataset[0:(len(dataset)-n)]
    return data

def copy_val(x):
    '''
    to copy a list or array in a new memory 
    without reference 
    x: list or array
    '''
    y =[]
    for i in x:
        y.append(i)
    return np.array(y)
## BENCHMARK ##
def smape(a, b):
    """
    Calculates sMAPE
    :param a: actual values
    :param b: predicted values
    :return: sMAPE
    """
    a = np.reshape(a, (-1,))
    b = np.reshape(b, (-1,))
    return np.mean(2.0 * np.abs(a - b) / (np.abs(a) + np.abs(b))).item() 

##===Mean Absolute Scaled Error ====##
def mase(insample, y_test, y_hat_test, freq):
    """
    Calculates MAsE
    :param insample: insample data
    :param y_test: out of sample target values
    :param y_hat_test: predicted values
    :param freq: data frequency
    :return:
    """
    y_hat_naive = []
    for i in range(freq, len(insample)):
        y_hat_naive.append(insample[(i - freq)])
    masep = np.mean(abs(insample[freq:] - y_hat_naive))
    return np.mean(abs(y_test - y_hat_test)) / masep
#Hier wird die "Time serie" als "spervised learning Problem" umgewandel.
#Die Datenmenge der Zeitreihen wird in Training und Testing Datamenge und jeweils in input & output Daten

# Hilfsfunktion , die eine Datenmenge in input und output Menge aufteile 
def split_input_output(dataset: np.ndarray, in_back: int=1) -> (np.ndarray, np.ndarray):
    """ 
    The function takes two arguments: the `dataset`, which is a NumPy array that we want to convert into a dataset,
    and the `in_back`, which is the number of previous time steps to use as input variables
    to predict the next time period — in this case defaulted to 1.
    :dataset: numpy dataset
    :in_variable: number of previous time steps as int
    :return: tuple of input and output dataset
    """
    Input, Output = [], []
    for i in range(len(dataset)-in_back):
        a = dataset[i:(i+in_back)]
        Input.append(a)
        Output.append(dataset[i + in_back])
    return np.array(Input), np.array(Output)

## Folgende Funktion split die Datenmende in Training and Testing Daten.

def split_into_train_test(dataset: np.ndarray,train_size, in_back) -> (np.ndarray, np.ndarray):
    """
    Splits dataset into training and test datasets. 
    : dataset: (np.ndarray) Time serie Dataset 
    : train_size: (int) Größe der Training Datamenge
    : look_back: (int) number of previous time steps 
    :return: tuple of training data and test dataset
    """
    if not train_size > in_back:
        raise ValueError('train_size muss größer als look_back',"train_size:",train_size,"in_back:",in_back)
    train= dataset[0:train_size]
    test = dataset[train_size - in_back:len(dataset)]
    #print('train_dataset: {}, test_dataset: {}'.format(len(train), len(test)))
    return train, test

## Es wird hier die Datenmenge in X_train,Y_train für das Training und X_test,Y_test für das Testing 

def all_split (dataset: np.ndarray,fh, in_back) -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    """
    Splits dataset into input-training (X_train), outout_training(Y_train) and input_test(X_test) , output_test(Y_test) datasets.
    : dataset:(np.ndarray) Time serie Dataset
    :df:(float64) Größe der Testing Datamenge 
    : in_back: (int) number of previous time steps 
    :return: x_train, y_train, x_test, y_test
    """
    #if not (size_prozent>0 and size_prozent<1):
        #raise ValueError('size_prozent of training must be in the interval 0 and 1')
    train_size = len(dataset)-fh
    training, testing = split_into_train_test(dataset,train_size,in_back)
    X_train, Y_train = split_input_output(training,in_back)
    X_test, Y_test = split_input_output(testing,in_back)
    return X_train,Y_train,X_test[0].reshape(1,-1),Y_test

def check_pred (dataset: pd.DataFrame,y_pred: np.ndarray):
    ''''
    this function check the negativity of the predicted values, set them to null 
    if they are negativ and to max value of the serie data if they are extrem high
    : dataset: Dataset of the serie
    : y_pred:  The list of predicted values
    : return:
    '''
    for i in range(len(y_pred)):
        if y_pred[i]<0:
            y_pred[i]=0
        if y_pred[i]> (9000*max(dataset)):
            y_pred[i]=max(dataset)
            

def LinReg_mod(x_train, y_train, x_test,forcast_horizon):
    '''
    This function return the prediction of x_test with 
    the model 'linear Regression'. The parameter 'normalize' of the model will be set on True
    : x_train: Input Training Dataset
    : y_train: Outpout Training Dataset
    : x_test: Input Testing Dataset
    : forcast_horizon: the horizon of forcasting
    '''
    y_predict = []
    # load of the model and setting of parameters
    model = LinearRegression(normalize=True)   #ARDRegression(compute_score=True)
    # Training of the model with x_train and y_train
    model.fit(x_train,y_train)
    # prediction of the model
    prediction_current = model.predict(x_test)[0]
    # Techniques of Iteration for the horizon forcasting
    for i in range(0, forcast_horizon):
        # add the first prediction to y_predict
        y_predict.append(prediction_current)
        # move the first element in x_test to the last position, in order to remove 
        x_test[0] = np.roll(x_test[0], -1)
        # set now the current_prediction value at the last position of x_test
        x_test[0][(len(x_test[0]) - 1)] = prediction_current
        prediction_current = model.predict(x_test)[0]
    return  np.asarray(y_predict) 

def Dtree_Reg  (x_train, y_train, x_test, fh):
    '''
    This function return the prediction of x_test with 
    the model 'Decision Tree Regression'. 
    : x_train: Input Training Dataset
    : y_train: Outpout Training Dataset
    : x_test: Input Testing Dataset
    : fh: the horizon of forcasting
    '''
    y_predict = []
    # load of the model and setting of parameters
    model = DecisionTreeRegressor(criterion='mae',max_depth=28,min_samples_split=5,
                                 min_samples_leaf =5)
    # Training of the model with x_train and y_train
    model.fit(x_train,y_train)
    # prediction of the model
    prediction_current = model.predict(x_test)[0]
    # Techniques of Iteration for the horizon forcasting
    for i in range(0, fh):
        # add the first prediction to y_predict
        y_predict.append(prediction_current)
        # move the first element in x_test to the last position, in order to remove 
        x_test[0] = np.roll(x_test[0], -1)
        # set now the current_prediction value at the last position of x_test
        x_test[0][(len(x_test[0]) - 1)] = prediction_current
        prediction_current = model.predict(x_test)[0]
    return  np.asarray(y_predict) 

def K_neaReg (x_train, y_train, x_test, fh):
    
    '''
    This function return the prediction of x_test with 
    the model 'k-nearest neighbours'. 
    : x_train: Input Training Dataset
    : y_train: Outpout Training Dataset
    : x_test: Input Testing Dataset
    : forcast_horizon: the horizon of forcasting
    '''
    
    y_predict = []
    # load of the model and setting of parameters
    model = KNeighborsRegressor(n_neighbors=5)
    # Training of the model with x_train and y_train
    model.fit(x_train,y_train)
    # prediction of the model
    prediction_current = model.predict(x_test)[0]
    # Techniques of Iteration for the horizon forcasting
    for i in range(0, fh):
        # add the first prediction to y_predict
        y_predict.append(prediction_current)
        # move the first element in x_test to the last position, in order to remove 
        x_test[0] = np.roll(x_test[0], -1)
        # set now the current_prediction value at the last position of x_test
        x_test[0][(len(x_test[0]) - 1)] = prediction_current
        prediction_current = model.predict(x_test)[0]
    return  np.asarray(y_predict) 

def SVM_Reg (x_train, y_train, x_test, fh):
    
    '''
    This function return the prediction of x_test with 
    the model 'Support Vector Regression'. 
    : x_train: Input Training Dataset
    : y_train: Outpout Training Dataset
    : x_test: Input Testing Dataset
    : fh: the horizon of forcasting
    '''
    y_predict = []
    # load of the model and setting of parameters
    model =  SVR(kernel='rbf', C=1e3, gamma=0.1)
    # Training of the model with x_train and y_train
    model.fit(x_train,y_train)
    # prediction of the model
    prediction_current = model.predict(x_test)[0]
    # Techniques of Iteration for the horizon forcasting
    for i in range(0, fh):
        # add the first prediction to y_predict
        y_predict.append(prediction_current)
        # move the first element in x_test to the last position, in order to remove 
        x_test[0] = np.roll(x_test[0], -1)
        # set now the current_prediction value at the last position of x_test
        x_test[0][(len(x_test[0]) - 1)] = prediction_current
        prediction_current = model.predict(x_test)[0]
    return  np.asarray(y_predict)     



In [26]:

def CV_Evaluation (dataset: pd.DataFrame,fh,in_back,freq):
    '''
    
    '''
    
    # number of Split
    f = int((dataset.shape[0]/(fh+1))) 
    
    # check of the number of split
    if f <= 1:
        raise ValueError("Split is 1 or 0",f)
    
    # Time Series cross-validator of sckitlearn
    splits = TimeSeriesSplit(n_splits=f)
    
    index = 1

    MASE = []    # a list to save all mase_value of each Split
    sMAPE = []    # a list to save all smape_value of each Split

    # Iteration that generates the index of each split
    for train_index, test_index in splits.split(dataset):
        
        # The merge of the generate Split
        train = dataset[train_index]
        test = dataset[test_index]
        frames = [train,test]     
        SPLIT = pd.concat(frames)
        print ("Split Nr:", index)
        print("Anzahl Observations: ", len(train)+len(test))
        print("train Observations: ", len(train))
        print("test Observations: ", len(test),"\n\n")
        # the Split will be splitted into x_train,y_train,x_test,y_test
        x_train,y_train,x_test,y_test =  all_split(SPLIT,fh,in_back)
        
        #Prediction of the model
        Y_pred_LinReg = Dtree_Reg(x_train, y_train, copy_val(x_test),fh)
        
        # Check of the 
        check_pred(dataset,Y_pred_LinReg)
        
        # The value of the metrics will be calculated and added to the list
        sMAPE.append(smape(y_test, Y_pred_LinReg))
        MASE.append(mase(train, y_test, Y_pred_LinReg, 1))    
        
        index += 1
     
    #print("MASE:  ",np.mean(MASE))    
    #print("sMAPE:  ",np.mean(sMAPE)) 
    return np.mean(MASE), np.mean(sMAPE)
CV_Evaluation(remov_nan(Dataset_weekly.iloc[:,5]),13,5,1)

Split Nr: 1
Anzahl Observations:  133
train Observations:  120
test Observations:  13 


Split Nr: 2
Anzahl Observations:  146
train Observations:  133
test Observations:  13 


Split Nr: 3
Anzahl Observations:  159
train Observations:  146
test Observations:  13 


Split Nr: 4
Anzahl Observations:  172
train Observations:  159
test Observations:  13 


Split Nr: 5
Anzahl Observations:  185
train Observations:  172
test Observations:  13 


Split Nr: 6
Anzahl Observations:  198
train Observations:  185
test Observations:  13 


Split Nr: 7
Anzahl Observations:  211
train Observations:  198
test Observations:  13 


Split Nr: 8
Anzahl Observations:  224
train Observations:  211
test Observations:  13 


Split Nr: 9
Anzahl Observations:  237
train Observations:  224
test Observations:  13 


Split Nr: 10
Anzahl Observations:  250
train Observations:  237
test Observations:  13 


Split Nr: 11
Anzahl Observations:  263
train Observations:  250
test Observations:  13 


Split Nr: 12
Anzahl

Split Nr: 92
Anzahl Observations:  1316
train Observations:  1303
test Observations:  13 


Split Nr: 93
Anzahl Observations:  1329
train Observations:  1316
test Observations:  13 


Split Nr: 94
Anzahl Observations:  1342
train Observations:  1329
test Observations:  13 


Split Nr: 95
Anzahl Observations:  1355
train Observations:  1342
test Observations:  13 


Split Nr: 96
Anzahl Observations:  1368
train Observations:  1355
test Observations:  13 


Split Nr: 97
Anzahl Observations:  1381
train Observations:  1368
test Observations:  13 


Split Nr: 98
Anzahl Observations:  1394
train Observations:  1381
test Observations:  13 


Split Nr: 99
Anzahl Observations:  1407
train Observations:  1394
test Observations:  13 


Split Nr: 100
Anzahl Observations:  1420
train Observations:  1407
test Observations:  13 


Split Nr: 101
Anzahl Observations:  1433
train Observations:  1420
test Observations:  13 


Split Nr: 102
Anzahl Observations:  1446
train Observations:  1433
test Observ

(1.6292404861275234, 0.3398820902333456)

In [ ]:
def main_prediction_CV (Data: pd.DataFrame,fh,freq):
    
    
    n = Data.shape[1]   # number of Serie in the Dataset
    
    Lin_Reg_MASE =[]     # a list to save all mase_values of linear regression of each Serie forcasting 
    Lin_Reg_sMAPE =[]    # a list to save all smape_values of linear regression of each Serie forcasting
    
   
    c =0
    # Iteration through each serie in the dataset
    for i in range(n):
        zr = Data.iloc[:,i]
        
        # remove all NaN value from the serie
        new_Data = remov_nan (zr)
        
        in_back = int(0.065*len(new_Data))
        # load the value of new_Data
        
        
        Mase,sMape = CV_Evaluation(new_Data,fh,in_back,freq)
        # calculation of Error
        Lin_Reg_sMAPE.append(sMape)

        Lin_Reg_MASE.append(Mase)
        
        #print(c)
        c+=1
        
    # Printing
    print("number of Time Series:",c)
    
    print("\n---------FINAL RESULTS---------")
    
    print("===========  sMAPE  =============\n")
    
    print("#### Dec_Tree ####\n", np.mean(Lin_Reg_sMAPE), "\n")

    print("============  MASE  =============")
    
    print("#### Dec_Tree ####\n", np.mean(Lin_Reg_MASE), "\n\n")
    

In [ ]:
# Weekly Data
main_prediction_CV(Dataset_weekly,13,1) 

In [20]:
# Hourly Data
main_prediction_CV(Dataset_weekly,13,1) 

In [52]:
#  daily Data
main_prediction_CV(Dataset_weekly,13,1) 

In [54]:
#  monthly Data
main_prediction_CV(Dataset_weekly,13,1) 

In [ ]:
#  quaterly Data
main_prediction_CV(Dataset_weekly,13,1) 

In [ ]:
#  yearly Data
main_prediction_CV(Dataset_weekly,13,1) 

In [ ]:
def main_prediction_CV_ (Data: pd.DataFrame,fh,freq):
    
    
    n = Data.shape[1]   # number of Serie in the Dataset
    
    Lin_Reg_MASE =[]     # a list to save all mase_values of linear regression of each Serie forcasting 
    Lin_Reg_sMAPE =[]    # a list to save all smape_values of linear regression of each Serie forcasting
    
   
    c =0
    # Iteration through each serie in the dataset
    for i in range(n):
        zr = Data.iloc[:,i]
        
        # remove all NaN value from the serie
        new_Data = remov_nan (zr)
        
        in_back = int(0.065*len(new_Data))
        # load the value of new_Data
        
        
        Mase,sMape = CV_Evaluation(new_Data,fh,in_back,freq)
        # calculation of Error
        Lin_Reg_sMAPE.append(sMape)

        Lin_Reg_MASE.append(Mase)
        
        #print(c)
        c+=1
        
    # Printing
    print("number of Time Series:",c)
    
    print("\n---------FINAL RESULTS---------")
    
    print("===========  sMAPE  =============\n")
    
    print("#### Lin_Reg ####\n", np.mean(Lin_Reg_sMAPE), "\n")

    print("============  MASE  =============")
    
    print("#### Lin_Reg ####\n", np.mean(Lin_Reg_MASE), "\n\n")